## Setup

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import itertools
from collections import Counter
from networkx.drawing.nx_agraph import graphviz_layout
from skbio.stats.composition import ilr
from skbio.stats.composition import clr
from skbio.stats.composition import multiplicative_replacement
import seaborn as sns
from matplotlib import rcParams
import os
from tqdm import tqdm
sns.set()
sns.set(font_scale=1.5)

## Load PCors and Tables

In [2]:
partials0 = []
partials1 = []

bootstrap_n = 100

for i in range(bootstrap_n):
    partials0.append(pd.read_csv("df0PCors/df0_PCor"+str(i)+".csv", header=0))
    partials1.append(pd.read_csv("df1PCors/df1_PCor"+str(i)+".csv", header=0))

In [3]:
tables0 = []
tables1 = []

for i in range(bootstrap_n):
    table0 = pd.read_csv("df0tables/df0_table"+str(i)+".csv", header=0)
    names = {}
    for c in table0.columns:
        names[c] = c.replace("-",";")
    table0.rename(names, axis=1, inplace=True)
    tables0.append(table0)

    table1 = pd.read_csv("df1tables/df1_table"+str(i)+".csv", header=0)
    names = {}
    for c in table1.columns:
        names[c] = c.replace("-",";")
    table1.rename(names, axis=1, inplace=True)
    tables1.append(table1)

taxonomy = pd.read_csv('data/taxonomy1.csv', header=0).set_index('Genera')

## Create Networks

In [4]:
#Takes in the partial correlation matrix pulls top triangle and returns associations that are not 0 or -1
def allAssociations(df):
    df2 = df.copy()
    df3 = df2.drop(columns=['Unnamed: 0']).copy()
    indexRename = {}
    count = 0
    
    for column in df3.columns:
        indexRename[count] = column
        count+=1
        
    df3.rename(index=indexRename, inplace=True)

    df4 = df3.where(np.triu(np.ones(df3.shape)).astype(bool))
    df4 = df4.stack().reset_index()
    df4.columns = ['Node1','Node2','Association_Weight']
    
    for i in df4.index:
        if df4.at[i,'Association_Weight'] == -1.0:
            df4.drop(index=i, inplace=True)
            
    return(df4)

In [5]:
associations0 = []
associations1 = []

for i in range(100):
    associations0.append(allAssociations(partials0[i]))
    associations1.append(allAssociations(partials1[i]))

In [6]:
associations0[0]

,Node1,Node2,Association_Weight
1,k__Archaea.p__Euryarchaeota.c__Methanobacteria...,k__Bacteria.p__Actinobacteria.c__Actinobacteri...,0.0
2,k__Archaea.p__Euryarchaeota.c__Methanobacteria...,k__Bacteria.p__Actinobacteria.c__Actinobacteri...,0.0
3,k__Archaea.p__Euryarchaeota.c__Methanobacteria...,k__Bacteria.p__Actinobacteria.c__Actinobacteri...,0.0
4,k__Archaea.p__Euryarchaeota.c__Methanobacteria...,k__Bacteria.p__Actinobacteria.c__Actinobacteri...,0.0
5,k__Archaea.p__Euryarchaeota.c__Methanobacteria...,k__Bacteria.p__Actinobacteria.c__Actinobacteri...,0.0
...,...,...,...
4648,k__Bacteria.p__Synergistetes.c__Synergistia.o_...,k__Bacteria.p__Synergistetes.c__Synergistia.o_...,0.0
4649,k__Bacteria.p__Synergistetes.c__Synergistia.o_...,k__Bacteria.p__Verrucomicrobia.c__Verrucomicro...,0.0
4651,k__Bacteria.p__Synergistetes.c__Synergistia.o_...,k__Bacteria.p__Synergistetes.c__Synergistia.o_...,0.0
4652,k__Bacteria.p__Synergistetes.c__Synergistia.o_...,k__Bacteria.p__Verrucomicrobia.c__Verrucomicro...,0.0


In [7]:
import networkx as nx
import numpy as np
from scipy import integrate

def createSparseGrowthNetworkSpecies(partialdf, abundanceDF):
    G = nx.Graph()
       
    for node in list(set(list(partialdf['Node1']) + list(partialdf['Node2']))):
        nodename = node.replace(".",";").strip().split(";")
        family = ""
        for n in nodename:
            if "f__" in n:
                family = n
        if isinstance(family, pd.Series):
            family = family.iloc[0].strip()
        G.add_node(node.replace(".",";").strip(), relativeAbundance=abundanceDF[node.replace(".",";").strip()].mean(), family=family)
        # , genera=generaFindDF_master.at[node.replace(".","-").strip(), 'Genus'].strip())
    
    for row in partialdf.index:
        if abs(partialdf.at[row, 'Association_Weight']) > 0.01:
            G.add_edge(partialdf.at[row, 'Node1'].replace(".",";").strip(),partialdf.at[row,'Node2'].replace(".",";").strip(), weight=partialdf.at[row, "Association_Weight"])
        else:
            continue
        
    return G;

In [8]:
networks0 = []
networks1 = []

import logging

for i in range(100):
    networks0.append(createSparseGrowthNetworkSpecies(associations0[i], tables0[i]))
    networks1.append(createSparseGrowthNetworkSpecies(associations1[i], tables1[i]))

## Network Properties

In [20]:
from networkx.algorithms.community.quality import modularity

def emergentNetworkProperties(network1, cohortName, partitions):
    columnNames = {"Network":[],  "Nodes":[], "Edges":[], "Positive_Edges":[], "Negative_Edges":[], "Nodes_in_Largest_Component":[], "Single_Nodes":[], "Density":[], "Average_Degree":[], "Average_Betweenness":[], "Modularity":[], "Connectedness":[], "ASPL":[], "Family_Assortativity":[], "Degree_Assortativity":[], "Transitivity":[]}
    ################################################
    columnNames['Network'].append(cohortName)
    columnNames['Nodes'].append(len(network1.nodes))
    columnNames['Edges'].append(len(network1.edges))
    ################################################
    positiveEdges = []
    negativeEdges = [] 
    
    weights = nx.get_edge_attributes(network1, 'weight')
    
    for edge in list(network1.edges):
        if float(weights[edge]) > 0.0:
            positiveEdges.append(weights[edge])               
        else:
            negativeEdges.append(weights[edge])
            
    columnNames['Positive_Edges'].append(len(positiveEdges))
    columnNames['Negative_Edges'].append(len(negativeEdges))
    
    #Find average degree of all nodes
    degrees = []
    for i in network1.nodes:
        degrees.append(network1.degree(i))
    avgDeg = np.mean(degrees)
    columnNames['Average_Degree'].append(avgDeg)

    # Betweenness Centrality
    betweenness = nx.betweenness_centrality(network1, weight='weight')
    avgBet = np.mean(list(betweenness.values()))
    columnNames['Average_Betweenness'].append(avgBet)
    
    #Find Average Shortest Path Length (Average of over all connected components)
    Gcc = sorted(nx.connected_components(network1), key=len, reverse=True)
    L2 = []
    for g in Gcc:
        if len(g) > 1:
            G0 = network1.subgraph(g)
            L2.append(nx.average_shortest_path_length(G0))
        else:
            continue
            weight=None
            
    columnNames['ASPL'] = np.mean(L2)
    
    columnNames['Modularity'].append(modularity(network1, partitions))
    
    #Find out how many nodes have no connections
    counttt = 0
    for i in list(nx.connected_components(network1)):
        if len(i) >1:
            continue
        else:
            counttt+=1
    columnNames['Single_Nodes'] = counttt
    
    #Find how many nodes are in the biggest component of the graph
    networkComponents = sorted(nx.connected_components(network1), key=len, reverse=True)
    bigcomponent = network1.subgraph(networkComponents[0])
    columnNames['Nodes_in_Largest_Component'] = len(list(nx.connected_components(bigcomponent))[0])
    
    
    ###################################################################################
    #Some statistics only work on connected components. Check if the network is connected.
    if nx.is_connected(network1):
        columnNames['Connectedness'].append("True")
        columnNames['Degree_Assortativity'].append(nx.degree_assortativity_coefficient(network1))
        columnNames['Transitivity'].append(nx.transitivity(network1))
        columnNames['Degree_Assortativity'].append(nx.degree_assortativity_coefficient(network1))
        columnNames['Family_Assortativity'].append(nx.attribute_assortativity_coefficient(network1, 'family'))
        columnNames['Transitivity'].append(nx.transitivity(network1))
        columnNames['Density'].append(nx.density(network1))
    else:
        columnNames['Connectedness'].append("False")
        columnNames['Degree_Assortativity'].append(nx.degree_assortativity_coefficient(network1))
        columnNames['Family_Assortativity'].append(nx.attribute_assortativity_coefficient(network1, 'family'))
        columnNames['Transitivity'].append(nx.transitivity(network1))
        columnNames['Density'].append(nx.density(network1))
        
    cohortDF = pd.DataFrame(columnNames)
    return(cohortDF)

In [18]:
partitions0 = []
partitions1 = []

from networkx.algorithms.community.label_propagation import asyn_lpa_communities
from networkx.algorithms.community import greedy_modularity_communities

for i in range(bootstrap_n):
    partitions0.append(list(greedy_modularity_communities(networks0[i], weight='weight')))
    partitions1.append(list(greedy_modularity_communities(networks1[i], weight='weight')))

# for i in range(bootstrap_n):
#     print(i)
#     partitions0.append(list(asyn_lpa_communities(networks0[i], weight='weight')))
#     partitions1.append(list(asyn_lpa_communities(networks1[i], weight='weight')))

In [21]:
networkProperties0 = []
networkProperties1 = []
networkProperty = []

for i in range(bootstrap_n):
    networkProperties0.append(emergentNetworkProperties(networks0[i], "Healthy", partitions0[i]).set_index("Network"))
    networkProperties1.append(emergentNetworkProperties(networks1[i], "Schizophrenic", partitions1[i]).set_index("Network"))
    networkProperty.append(pd.concat([networkProperties0[i], networkProperties1[i]]))

In [22]:
networkProperties0[0]

,Nodes,Edges,Positive_Edges,Negative_Edges,Nodes_in_Largest_Component,Single_Nodes,Density,Average_Degree,Average_Betweenness,Modularity,Connectedness,ASPL,Family_Assortativity,Degree_Assortativity,Transitivity
Network,,,,,,,,,,,,,,,
Healthy,96,92,74,18,28,47,0.020175,1.916667,0.003642,0.720778,False,1.440873,0.081902,0.674733,0.661509


## Cliques

In [ ]:
cliques0 = []
cliques1 = []

for i in range(bootstrap_n):
    cliques0.append(list(nx.find_cliques(networks0[i])))
    cliques1.append(list(nx.find_cliques(networks1[i])))

In [ ]:
def returnGoodCliques(network, cliqueSize):
    cliques = []
    for i in list(nx.enumerate_all_cliques(network)):
        if len(i) == cliqueSize:
            cliques.append(i)
    return(cliques)

In [ ]:
# Finds the maximal cliques for all networks
def findCliques(network1, network2, name1, name2, cliqueSize):
    
    names = []
    
    allCliques_temp1 = []
    allCliques_temp2 = []

    netNames = {}
    
    if network1 != 'No':
        allCliques_temp1 = returnGoodCliques(network1, cliqueSize)
        names.append(name1)
        netNames[name1]=allCliques_temp1
        
    if network2 != 'No':
        allCliques_temp2 = returnGoodCliques(network2, cliqueSize)    
        names.append(name2)
        netNames[name2]=allCliques_temp2
        
    
    # Set one cohort as all cliques so far then add cliques that are unique from each cohort
    allCliques = allCliques_temp1.copy()

    for i in allCliques_temp2:
        
        countFlag = False
        
        for x in allCliques:
            if sorted(i) == sorted(x):
                countFlag = True
            else:
                continue
                
        if countFlag == False:
            allCliques.append(sorted(i))
        else:
            continue
    
    ####################################################################################
    # BUILD NEW DATAFRAME AND SAY IF ITS IN THAT COHORT OR NOT
    
    clidDic = {"Clique":[], "Healthy":[], "Schizophrenic":[]}
   
    for clique in allCliques:
        
        clidDic['Clique'].append(clique)
        clique1count = False
        clique2count = False
        
        for clique1 in allCliques_temp1:
            
            if sorted(clique1) == sorted(clique):
                clique1count = True
            else:
                continue
                
        if clique1count == True:
            clidDic['Healthy'].append("Present")
        else:
            clidDic['Healthy'].append(np.nan)
        
        ###############################################
        
        for clique2 in allCliques_temp2:
            if sorted(clique2) == sorted(clique):
                clique2count = True
            else:
                continue
                
        if clique2count == True:
            clidDic['Schizophrenic'].append("Present")
        else:
            clidDic['Schizophrenic'].append(np.nan)
            
        ################################################
        

    cliqueDF = pd.DataFrame(data=clidDic).set_index("Clique")
    
    return(cliqueDF)

In [ ]:
allCliques = []

for i in range(bootstrap_n):
    allCliques.append(findCliques(networks0[i], networks1[i], 'Healthy', 'Schizophrenic', 3))

## t-test?

In [ ]:
from scipy import stats

metric_names = list(networkProperty[0].columns)
metric_names.remove("Connectedness")
metrics0 = {key: [] for key in metric_names}
metrics1 = {key: [] for key in metric_names}

for i in range(bootstrap_n):
    for metric in metric_names:
        metrics0[metric].append(networkProperties0[i][metric][0])
        metrics1[metric].append(networkProperties1[i][metric][0])

print(sorted(metrics0['Average_Degree']))
print(sorted(metrics1['Average_Degree']))

pvals = {key: None for key in metric_names}
for metric in metric_names:
    pvals[metric] = []
    pvals[metric].append(np.mean(np.array(metrics0[metric])))
    pvals[metric].append(np.mean(np.array(metrics1[metric])))
    pvals[metric].append(stats.ttest_ind(np.array(metrics0[metric]), np.array(metrics1[metric]))[1])
comparison = pd.DataFrame(data=pvals, index=["Healthy Mean", "Schizophrenic Mean", "p-value"])
comparison